# Maximal Clique Problem
## Report of implementations
**Student:** Matheus Silva Melo de Oliveira

**Institution:** Insper Instituto de Ensino e Pesquisa

<div align="center">
<img width="400px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d0/VR_complex.svg/1200px-VR_complex.svg.png"/>
</div>

## Motivation
The concept of a clique in a graph is relatively simple. At the most general level, a clique is a subset of a network in which the actors are more closely connected to each other than to other members of the network. In terms of friendship ties, for example, it is not uncommon to find human groups that form cliques based on age, gender, race, ethnicity, religion, ideology, and many other factors. A clique is, therefore, a set of vertices in a graph where every pair of vertices is directly connected by an edge.

Finding the maximum clique in a graph is a computationally challenging task due to the combinatorial nature of the problem. The computational difficulty arises from the need to explore all possible combinations of vertices to identify the largest clique, which becomes exponential in relation to the number of vertices. This results in high computational complexity, even for moderately large graphs.

Cliques are important because, in addition to developing homogeneous behaviors among their members, they have, by definition, great proximity, increasing the speed of exchanges. Thus, information directed at a clique is quickly absorbed by its members, who tend to perceive it similarly. This is important, for example, in segmentation strategies.

Therefore, the efficient resolution of the maximum clique problem has valuable applications in areas ranging from computer science to data analysis in social networks.



**It's expected to run  this notebook in Google Colab, to run in a local machine (Linux ou WSL), run *run.sh* file**

## Setup

In [ ]:
%cd "../.."
!rm -rf Maximal-Clique-Problem

/content


In [ ]:
!git clone https://github.com/matheus-1618/Maximal-Clique-Problem

Cloning into 'Maximal-Clique-Problem'...
remote: Enumerating objects: 100, done.
remote: Counting objects: 100% (100/100), done.
remote: Compressing objects: 100% (76/76), done.
remote: Total 100 (delta 48), reused 64 (delta 22), pack-reused 0
Receiving objects: 100% (100/100), 1.13 MiB | 3.67 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
%cd "Maximal-Clique-Problem/src"

/content/Maximal-Clique-Problem/src


## Generating the graph
Using the python lib *networkx*, we create a graph that it's going to be used in the implementations.

In [ ]:
!python3 python/generate_graph.py

Grafo densamente conectado gerado e salvo em 'implementations/graph.txt'.


## First implementation: Simple Search clique
In the file named **0_search_clique.cpp** , we developed a simple search algorithm that starts always from the last node (by its ID in the graph file), and add a adjacent node if both belongs to the same clique.

In [ ]:
!g++ -Wall -O3 -g implementations/0_search_clique.cpp -o 0_search_clique
!./0_search_clique

[Implementation-Basics] Clique's Size: 9 Maximal Clique: 27 29 60 64 79 95 96 97 100 


## Second implementation: Using the number of edges as heuristics
In the file named **1_edges_heuristic.cpp** ,we keep the basic logic of the implementation above, we sort the candidate's vector by the quantity of edges.

In [ ]:
!g++ -Wall -O3 -g implementations/1_edges_heuristic.cpp -o 1_edges_heuristic
!./1_edges_heuristic

[Implementation-Heuristics] Clique's Size: 13 Maximal Clique: 7 11 12 21 40 58 59 69 74 78 79 85 86 


## Third implementation: Recursive Approach
In the file named **2_recursive_clique.cpp**, we're using a recursive approach for each clique found.

In [ ]:
g++ -Wall -O3 -g implementations/2_recursive_clique.cpp -o 2_recursive_clique
./2_recursive_clique

## Checking if the obtained Cliques are the maximal one or in the group of Maximals
Using another python script from the lib *networkx*, we can check the maximal Clique and it's size, and compare with the result of our implementations above.  

In [ ]:
!python3 python/verify_clique.py

[Verification] Clique's Size: 15 Maximal Clique:  [6, 18, 29, 34, 39, 40, 70, 74, 78, 81, 83, 85, 86, 90, 97]


## Clean up

In [ ]:
%cd "../.."
!rm -rf Maximal-Clique-Problem